## Container Terminal Congestion Problem

Consider a container terminal with a storage yard consisting of 100 blocks, each with
storage space to hold 600 containers, numbered 1 to 100. 

For $i = 1, \dots, 100$, if no newly arriving containers are sent to block $i$, the number 
of stored containers is expected to be:

320, 157, 213, 96, 413, 312, 333, 472, 171, 222,
439, 212, 190, 220, 372, 101, 212, 251, 86, 79,
295, 138, 343, 281, 372, 450, 100, 183, 99, 505,
99, 254, 330, 279, 300, 150, 340, 221, 79, 119,
43, 71, 219, 363, 98, 500, 413, 259, 182, 391,
360, 447, 181, 233, 414, 30, 333, 427, 251, 83,
144, 404, 76, 84, 196, 336, 411, 280, 115, 200,
117, 284, 263, 477, 431, 297, 380, 327, 155, 360,
360, 290, 350, 157, 116, 141, 82, 116, 99, 78,
220, 182, 96, 301, 121, 278, 372, 119, 282, 310


The terminal estimates that **15,166 new containers** will be unloaded and dispatched 
to the storage yard in this period.  

**Goal:** Determine an optimal plan to allocate these new containers to blocks to 
**minimize congestion** on the terminal roads.

---

## Import the libraries

In [109]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.environ import ConcreteModel, Param, Var, RangeSet, Reals, Constraint, Objective, minimize, NonNegativeIntegers

## Model Inputs

In [57]:
# a_i: The number of stored containers in block i at the end of the period if no new containers are sent

initial_containers_list = [320, 157, 213, 96, 413, 312, 333, 472, 171, 222,
439, 212, 190, 220, 372, 101, 212, 251, 86, 79,
295, 138, 343, 281, 372, 450, 100, 183, 99, 505,
99, 254, 330, 279, 300, 150, 340, 221, 79, 119,
43, 71, 219, 363, 98, 500, 413, 259, 182, 391,
360, 447, 181, 233, 414, 30, 333, 427, 251, 83,
144, 404, 76, 84, 196, 336, 411, 280, 115, 200,
117, 284, 263, 477, 431, 297, 380, 327, 155, 360,
360, 290, 350, 157, 116, 141, 82, 116, 99, 78,
220, 182, 96, 301, 121, 278, 372, 119, 282, 310]

a_i = {i+1: val for i, val in enumerate(initial_containers_list)}

# B: The number of blocks in storage yard
# A: The number of storage space in each block

N_BLOCKS = 100
MAX_CAPACITY = 600

PERIOD = 4  # 4 hours

# N: The number of new containers being unloaded and dispatched to the storage yard
N_CONTAINERS = 15166


# Calculate Fill-Ration
FILL_RATIO = (N_CONTAINERS + sum(a_i[i] for i in a_i)) / (MAX_CAPACITY * N_BLOCKS)

## LP with Pyomo

In [61]:
model = ConcreteModel()

model.blocks = pyo.Set(initialize=a_i.keys())

model.x = pyo.Var(model.blocks, domain=pyo.NonNegativeReals)
model.u_plus = pyo.Var(model.blocks, domain=pyo.NonNegativeReals)
model.u_minus = pyo.Var(model.blocks, domain=pyo.NonNegativeReals)

def fill_ratio_rule(model, i):
    return a_i[i] + model.x[i] - (model.u_plus[i] - model.u_minus[i]) == MAX_CAPACITY * FILL_RATIO
model.fill_ratio_con = pyo.Constraint(model.blocks, rule=fill_ratio_rule)

def total_allocation_rule(model):
    return sum(model.x[i] for i in model.blocks) == N_CONTAINERS
model.total_allocation_con = pyo.Constraint(rule=total_allocation_rule)

model.pprint()


1 Set Declarations
    blocks : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  100 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}

3 Var Declarations
    u_minus : Size=100, Index=blocks
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : False :  Tr

In [62]:
solver = pyo.SolverFactory('glpk')  
results = solver.solve(model, tee=True)

for i in model.blocks:
    print(f"x[{i}] = {pyo.value(model.x[i])}, u+[{i}] = {pyo.value(model.u_plus[i])}, u-[{i}] = {pyo.value(model.u_minus[i])}")

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\pln01\AppData\Local\Temp\tmp9h2y1dq1.glpk.raw --wglp C:\Users\pln01\AppData\Local\Temp\tmpve3glxz2.glpk.glp
 --cpxlp C:\Users\pln01\AppData\Local\Temp\tmpr8yq4sm1.pyomo.lp
Reading problem data from 'C:\Users\pln01\AppData\Local\Temp\tmpr8yq4sm1.pyomo.lp'...
101 rows, 301 columns, 400 non-zeros
1014 lines were read
Writing problem data to 'C:\Users\pln01\AppData\Local\Temp\tmpve3glxz2.glpk.glp'...
908 lines were written
GLPK Simplex Optimizer, v4.65
101 rows, 301 columns, 400 non-zeros
Preprocessing...
87 rows, 185 columns, 271 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 87
      0: obj =   1.000000000e+00 inf =   1.581e+04 (86)
     87: obj =   1.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.2 Mb (23090

# Solve with Combinatorial Optimization

In [ ]:
comb_dict = {a_i: 0 for a_i in sorted(initial_containers_list, reverse=False)}

comb_keys = list(comb_dict.keys())

comb_dict[comb_keys[0]] = min(N_CONTAINERS, max(0, MAX_CAPACITY * FILL_RATIO - comb_keys[0]))

for i in range(1, len(comb_keys)):
    comb_dict[comb_keys[i]] = min(max(0, MAX_CAPACITY * FILL_RATIO - comb_keys[i]), N_CONTAINERS - sum(comb_dict[comb_keys[j]] for j in range(1, i)))

In [115]:
sorted_blocks = sorted(initial_containers_list)

decision_output = pd.DataFrame({
    "Stored Containers": sorted_blocks,
    "New Containers": [int(comb_dict[a_i]) for a_i in sorted_blocks]
})

decision_output.to_csv("Decision_Output.csv")

decision_output

,Stored Containers,New Containers
0,30,366
1,43,353
2,71,325
3,76,320
4,78,318
...,...,...
95,450,0
96,472,0
97,477,0
98,500,0


# Output Analysis: Yard Fill-Ratio Allocation LP

The Pyomo LP model was solved to determine allocations $x_i$ to each block in the storage yard, with the goal of balancing fill ratios across all blocks at the end of the period. The decision variables and slacks are defined as:  

- $x_i$ → allocation to block $i$ in the current period  
- $u_i^+$ → positive deviation from the target fill $A \cdot F$ (block overfilled)  
- $u_i^-$ → negative deviation from the target fill $A \cdot F$ (block underfilled)  

---

## 1. General Observations

- **Majority of blocks** have $u_i^+ = 0$ and $u_i^- = 0$, indicating that they **exactly meet the target fill**.  
- **Blocks with $u_i^+ > 0$** indicate overfilled blocks (current allocation exceeds target).  
- **Blocks with $u_i^- > 0$** indicate underfilled blocks (current allocation is below target).  
- **Allocations $x_i = 0$** often correspond to blocks that are overfilled ($u_i^+ > 0$) or underfilled ($u_i^- > 0$), reflecting capacity or fill constraints.

---

## 2. Interpretation of Slacks

- **Overfilled blocks ($u_i^+ > 0$)**:  
  - Example: Block 8 has $u_8^+ = 75.21$  
  - Implication: allocation to this block exceeds target fill. Future allocation to these blocks should be reduced.  

- **Underfilled blocks ($u_i^- > 0$)**:  
  - Example: Block 96 has $u_{96}^- = 118.79$  
  - Implication: allocation to this block is below target. These blocks should be prioritized in future periods to improve balance.  

- **Zero-slack blocks ($u_i^+ = u_i^- = 0$)**:  
  - Example: Block 1 has $x_1 = 76.79$ with zero slacks  
  - Implication: allocation is optimal; the block meets the target fill.

---

## 3. Decision-Making Insights

1. **Identify priority blocks:** Focus on blocks with $u_i^- > 0$ to reduce underfill.  
2. **Avoid over-allocation:** Blocks with $u_i^+ > 0$ are already overfilled and should not receive additional resources.  
3. **Monitor overall yard balance:** Total slack $$\sum_i (u_i^+ + u_i^-)$$ measures deviation from uniform fill. A lower total slack indicates a more balanced yard.  
4. **Plan future allocations:** Use slack information to redistribute resources from overfilled to underfilled blocks in subsequent periods.  

---

## 4. Example Table

| Block | Allocation $x_i$ | Overfill $u_i^+$ | Underfill $u_i^-$ | Interpretation |
|-------|-----------------|-----------------|------------------|----------------|
| 1     | 76.79           | 0.0             | 0.0              | Exactly meets target |
| 5     | 0.0             | 16.21           | 0.0              | Overfilled, do not allocate more |
| 96    | 0.0             | 0.0             | 118.79           | Underfilled, prioritize allocation |

---

## 5. Conclusion

The LP solution provides a clear allocation plan that minimizes deviation from the target fill ratio across blocks. The slacks ($u_i^+$ and $u_i^-$) serve as diagnostic indicators, helping to:

- Identify underfilled or overfilled blocks  
- Adjust allocations in future periods  
- Maintain a balanced yard and optimize storage utilization
